In [ ]:
import pandas as pd

data_path = "perf_sheets/opinions_stories_qd_score.csv"
data = pd.read_csv(data_path, sep='\t', header=None)

# detect rows where the experiment groups are defined - will have NaNs or non-numeric values in columns other than the first
group_rows = data.apply(lambda x: pd.to_numeric(x, errors='coerce')).isna().all(axis=1)
group_names = data.loc[group_rows].iloc[:, 0].values

# initialize variables to keep track of the start and end rows for each group
sub_dfs = []
start_row = None
group_name = None

# loop through each row to find where each group starts and ends
for i, (is_group_row, first_cell) in enumerate(zip(group_rows, data.iloc[:, 0])):
    if is_group_row and first_cell != 'Type':  # start of a new group
        if start_row is not None:
            sub_dfs.append({
                'name': group_name,
                'data': data.iloc[start_row:i].copy()
            })
        start_row = i + 1  # include the 'Type' row as a normal row
        group_name = first_cell

# add the last group
if start_row is not None:
    sub_dfs.append({
        'name': group_name,
        'data': data.iloc[start_row:].copy()
    })

# remove sub-dataframes where the name is nan
cleaned_sub_dfs = [sub_df for sub_df in sub_dfs if pd.notna(sub_df['name'])]

# display
cleaned_group_names = [group['name'] for group in cleaned_sub_dfs]
cleaned_group_names, cleaned_sub_dfs[0]['data'].head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

def plot_perf(group_data, group_name, id):
    # skip the first row since it contains the header information
    group_data = group_data.iloc[1:]
    
    fig, ax = plt.subplots(figsize=(3.5, 1.8), gridspec_kw={'width_ratios': [1,], 'wspace': 0.05})

    ax.xaxis.grid(True, linestyle='--', linewidth=0.5, color='gray')
    ax.yaxis.grid(False)
    
    # for each method
    unique_colors = sns.color_palette("husl", len(group_data[0]))
    
    # plot mean and CI
    for i, (_, row) in enumerate(group_data.iterrows()):
        ax.errorbar(x=row[1], y=i, 
                         xerr=[[row[1] - row[3]],
                               [row[4] - row[1]]],
                         fmt='none', capsize=7, color=unique_colors[i], linewidth=10)
        ax.vlines(x=row[1], ymin=i - 0.3, ymax=i + 0.3, colors='black', linewidth=2)
    ax.set_xlabel('QD Score', fontsize=18)

    # display method labels only for first plot, hide for other plots, to show plots along same row
    if id == 0:
        ax.set_yticks(range(len(group_data[0])))
        ax.set_yticklabels(group_data[0])
        ax.yaxis.set_tick_params(length=0)
    else:
        ax.set_yticks([])
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    
    plt.title(f'{group_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

# convert relevant columns to numeric type for each sub-dataframe
for sub_df in cleaned_sub_dfs:
    for col in sub_df['data'].columns[1:5]:  # columns with stats values
        sub_df['data'][col] = pd.to_numeric(sub_df['data'][col], errors='coerce')

# plot for each domain
for i, sub_df in enumerate(cleaned_sub_dfs):
    plot_perf(sub_df['data'], sub_df['name'], i)